In [1]:
import os
from torchvision import datasets
from PIL import Image
import torchvision.transforms as transforms
import torch
import torchvision.models as models
import numpy as np

batch_size=20
num_workers=0

data_dir='/floyd/input/kaggle_skin_cancer/'
train_dir=os.path.join(data_dir, 'train/')
valid_dir=os.path.join(data_dir, 'valid/')

train_data_transform=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_data_transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data=datasets.ImageFolder(train_dir, transform=train_data_transform)
valid_data=datasets.ImageFolder(valid_dir, transform=test_data_transform)

train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         shuffle=True)
valid_loader=torch.utils.data.DataLoader(valid_data,
                                         batch_size=batch_size,
                                         num_workers=num_workers, 
                                         shuffle=True)

loaders_scratch={
    "train": train_loader,
    "valid": valid_loader,
}

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1=nn.Conv2d(3, 16, 3, padding=1)
        self.conv2=nn.Conv2d(16, 32, 3, padding=1)
        self.conv3=nn.Conv2d(32, 64, 3, padding=1)
        self.pool=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(64*28*28, 500)
        self.fc2=nn.Linear(500, 2)
        self.dropout=nn.Dropout(0.3)
    
    def forward(self, x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        x=self.pool(F.relu(self.conv3(x)))
        x=x.view(-1,64*28*28)
        x=self.dropout(x)
        x=F.relu(self.fc1(x))
        x=self.dropout(x)
        x=self.fc2(x)
        return x

model_scratch = Net()
print(model_scratch)

use_cuda = torch.cuda.is_available()
if use_cuda:
    model_scratch.cuda()

Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [3]:
import torch.optim as optim

criterion_scratch = nn.CrossEntropyLoss()
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.01, momentum=0.9)

In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output=model(data)
            loss=criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            if batch_idx % 100 == 99:
                print('Epoch %d, Batch %d loss: %.6f' %
                  (epoch, batch_idx + 1, train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            output=model(data)
            loss=criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        if valid_loss < valid_loss_min:
            torch.save(model.state_dict(), save_path)
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            valid_loss_min = valid_loss
            
    return model

In [5]:
model_scratch = train(30, loaders_scratch, model_scratch, optimizer_scratch, criterion_scratch, use_cuda, 'model_scratch.pt')

Epoch 1, Batch 100 loss: 0.605328
Epoch: 1 	Training Loss: 0.594873 	Validation Loss: 0.463266
Validation loss decreased (inf --> 0.463266).  Saving model ...
Epoch 2, Batch 100 loss: 0.532838
Epoch: 2 	Training Loss: 0.525955 	Validation Loss: 0.463167
Validation loss decreased (0.463266 --> 0.463167).  Saving model ...
Epoch 3, Batch 100 loss: 0.507710
Epoch: 3 	Training Loss: 0.497764 	Validation Loss: 0.429014
Validation loss decreased (0.463167 --> 0.429014).  Saving model ...
Epoch 4, Batch 100 loss: 0.429237
Epoch: 4 	Training Loss: 0.444384 	Validation Loss: 0.419116
Validation loss decreased (0.429014 --> 0.419116).  Saving model ...
Epoch 5, Batch 100 loss: 0.433814
Epoch: 5 	Training Loss: 0.439973 	Validation Loss: 0.404448
Validation loss decreased (0.419116 --> 0.404448).  Saving model ...
Epoch 6, Batch 100 loss: 0.441325
Epoch: 6 	Training Loss: 0.440385 	Validation Loss: 0.416013
Epoch 7, Batch 100 loss: 0.427040
Epoch: 7 	Training Loss: 0.424853 	Validation Loss: 0.38

In [6]:
model_scratch.load_state_dict(torch.load('model_scratch.pt'))

<All keys matched successfully>

In [7]:
running_corrects = 0
for batch_idx, (data, target) in enumerate(loaders_scratch['valid']):
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    outputs=model_scratch(data)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == target.data)
acc = running_corrects.double() / len(loaders_scratch['valid'].dataset)
print(acc)

tensor(0.8455, device='cuda:0', dtype=torch.float64)
